# Day 7: Amplification Circuit

## Part 1

In [6]:
from os import path
import itertools
from collections import defaultdict, deque

In [7]:
POSITION = 0
IMMEDIATE = 1
RELATIVE = 2

ADD = 1
MUL = 2
INPUT = 3
OUTPUT = 4
JUMP_TRUE = 5
JUMP_FALSE = 6
LESS_THAN = 7
EQUALS = 8
ADD_RELATIVE_BASE = 9
HALT = 99

READ = 0
WRITE = 1

OPS = {
    ADD: (READ,READ,WRITE),
    MUL: (READ,READ,WRITE),
    INPUT: (READ,),
    OUTPUT: (WRITE,),
    JUMP_TRUE: (READ,READ),
    JUMP_FALSE: (READ,READ),
    LESS_THAN: (READ,READ,WRITE),
    EQUALS: (READ,READ,WRITE),
    ADD_RELATIVE_BASE: (READ,),
    HALT: ()
}

In [17]:
class VM:
    def __init__(self,code,inp=None):
        self.mem=list(code)
        self.inp=deque([] if inp is None else inp)
        self.out=[]
    
    def __getitem__(self,index):
        return self.mem[index]
    
    def __setitem__(self,index,val):
        self.mem[index]=val
        
    def get_args(self, arg_kinds, modes):
        args = []*4
        
        for i, kind in enumerate(arg_kinds):
            a = self[self.ip + 1 + i]
            mode=modes%10
            modes//=10
            
            if mode == RELATIVE:
                a+= self.relative_base
                
            if mode in (POSITION, RELATIVE):
                if a < 0:
                    raise Exception(f"Invalid access to negative memory index {a}")
                elif a>= len(self.mem):
                    self.mem += [0] * ( a +1 -len(self.mem))
                    
                if kind == READ:
                    a = self[a]
                elif kind != WRITE:
                    raise Exception(f"Invalid arg kind:{kind}")
            elif mode == IMMEDIATE:
                if kind == WRITE:
                    raise exception(f"invalid arg mode for write arg: {mode}")
            else:
                raise Exception(f"Invalid mode: {mode}")
            
            args[i] = a
        return args
    
    def push_in(self,inp):
        self.inp.extend(inp)
        
    def pop_out(self):
        out=self.out
        self.out=[]
        return out
        
    def run(self):
        self.ip=0
        self.relative_base=0
        
        while self[self.ip] != HALT:
            instr = self[self.ip]
            op=instr%100
            modes=instr//100
            
            if op not in OPS:
                raise Exception(f"Unknown Opcode: {op}")
                
            arg_kinds = OPS[op]
            a, b, c, d = self.get_args(arg_kinds, modes)
            self.ip += 1+ len(arg_kinds)
            
            if op == IN:
                while not self.inp:
                    yield
                self[a]=self.inp.popleft()
            elif op == OUT:
                self.out.append(a)
            elif op == ADD:
                self[c] = a + b
            elif op == MULT:
                self[c] = a * b
            elif op == LESS_THAN:
                self[c] = 1 if a < b else 0
            elif op == EQUALS:
                self[c] = 1 if a == b else 0
            elif op == JUMP_TRUE:
                if a != 0:
                    self.ip= b
            elif op == JUMP_FALSE:
                if a == 0:
                    self.ip=b
            elif op == ADD_RELATIVE_BASE:
                self.relative_base += a
            else:
                raise exception(f"Unimplemented opcode: {op}")

class Scheduler:
    OUT = object()

    def __init__(self, vms):
        self.vms = vms
        self.connections = [[] for _ in vms]
        self.waiting = deque(range(len(self.vms)))
        self.gens = [vm.run() for vm in self.vms]
        self.out = []

    def connect(self, start, end):
        self.connections[start].append(end)

    def run(self):
        while self.waiting:
            curr = self.waiting.popleft()

            try:
                next(self.gens[curr])
                self.waiting.append(curr)
            except StopIteration:
                pass

            out = self.vms[curr].pop_out()
            for conn in self.connections[curr]:
                if conn is Scheduler.OUT:
                    self.out.extend(out)
                else:
                    self.vms[conn].push_in(out)

        return self.out


with open(path.join(path.dirname(__file__), "input.txt")) as f:
    code = list(map(int, f.readline().strip().split(",")))

    result = float("-inf")

    for perm in itertools.permutations(range(5)):
        vms = [VM(code, [p]) for p in perm]
        vms[0].push_in([0])

        sched = Scheduler(vms)
        for i in range(4):
            sched.connect(i, i + 1)
        sched.connect(4, Scheduler.OUT)

        out = sched.run()
        result = max(result, out[-1])

    print("Part 1:", result)

    result = float("-inf")

    for perm in itertools.permutations(range(5, 10)):
        vms = [VM(code, [p]) for p in perm]
        vms[0].push_in([0])

        sched = Scheduler(vms)
        for i in range(4):
            sched.connect(i, i + 1)
        sched.connect(4, 0)
        sched.connect(4, Scheduler.OUT)

        out = sched.run()
        result = max(result, out[-1])

    print("Part 2:", result)


NameError: name '__file__' is not defined

In [11]:
VM()

In [13]:
print (a)

[None, None, None, None]


In [1]:
import doctest
import operator

In [2]:
def get_opcode(x):
    '''
    int -> int
    
    Returns the opcode of an instruction
    '''
    return x%100

In [3]:
def get_posorval(instruction, param, num, myprog):
    '''
    int, int, int -> int
    
    instruction: up to 5 char int of 3 parameter or value + 2 digit opcode.
    param: Which of the 3 parameters we are calculating on
    num: value in the program we are checking
    
    '''
    param=param-1
    filled_instruction=str(instruction).zfill(5)
    if (filled_instruction[param] == "0" ):
        return myprog[myprog[num]]
    elif (filled_instruction[param] == "1" ):
        return myprog[num]
    else: 
        return None

In [4]:
def opcode(prog, phase, inputval):
    '''
    list -> list
    
    A turing tape like computing device
    
    >>> opcode([1,0,0,0,99])
    [2, 0, 0, 0, 99]
    >>> opcode([2,3,0,3,99]) 
    [2, 3, 0, 6, 99]
    >>> opcode([2,4,4,5,99,0])
    [2, 4, 4, 5, 99, 9801]
    >>> opcode([1,1,1,4,99,5,6,0,99])
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
    >>> opcode([4, 1, 99])
    1
    [4, 1, 99]
    >>> opcode([8,1,1,5,99,-1])
    [8, 1, 1, 5, 99, 1]
    >>> opcode([8,1,2,5,99,-1])
    [8, 1, 2, 5, 99, 0]
    >>> opcode([11008,1,1,5,99,-1])
    [11008, 1, 1, 1, 99, -1]
    >>> opcode([11108,1,2,5,99,-1])
    [11108, 1, 2, 0, 99, -1]
    '''
    i=0
    inputnum=0
    opcode='00'
    while (opcode != 99):
        instruction=prog[i]
        assignparam=int(str(instruction).zfill(5)[0])
        opcode=get_opcode(instruction)
        if (opcode == 1):
            if (assignparam == 0 ):
                prog[prog[i+3]]=get_posorval(instruction, 3, i+1, prog)+get_posorval(instruction, 2, i+2, prog)
            elif(assignparam == 1 ):
                prog[i+3]=get_posorval(instruction, 3, i+1, prog)+get_posorval(instruction, 2, i+2, prog)
            i=i+4
        elif (opcode == 2):
            if (assignparam == 0 ):
                prog[prog[i+3]]=get_posorval(instruction, 3, i+1, prog)*get_posorval(instruction, 2, i+2, prog)
            elif(assignparam == 1 ):
                prog[i+3]=get_posorval(instruction, 3, i+1, prog)*get_posorval(instruction, 2, i+2, prog)
            i=i+4
        elif (opcode == 3):
        # Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
            assignparam=(instruction//100)%10
            inputnum=inputnum+1
            # Disable blocking input for now.
            #input_int=int(input())
            if (inputnum == 1):
                input_int=phase
            elif(inputnum ==2 ):
                input_int=inputval
            if (assignparam == 0 ):
                prog[prog[i+1]]=input_int
            elif(assignparam == 1 ):
                prog[i+1]=input_int
            i=i+2
        elif (opcode == 4):
        # Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.
            assignparam=(instruction//100)%10
            if (assignparam == 0 ):
                output=prog[prog[i+1]]
                
            elif(assignparam == 1 ):
                output=prog[i+1]
            # Disable blocking input for now.

            #print(output)
            i=i+2
        elif (opcode == 5):
        # Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
            assignparam=(instruction//100)%10
            if (get_posorval(instruction, 3, i+1, prog) != 0 ):
                i = get_posorval(instruction, 2, i+2, prog)
            else:
                i=i+3
        elif (opcode == 6):
        # Opcode 6 is jump-if-false: 
        # if the first parameter is zero, 
        #     it sets the instruction pointer to the value from the second parameter. 
        #     Otherwise, it does nothing.
            assignparam=(instruction//100)%10
            if (get_posorval(instruction, 3, i+1, prog) == 0 ):
                i = get_posorval(instruction, 2, i+2, prog)
            else:
                i=i+3
        elif (opcode == 7):
        # Opcode 7 is less than: 
        # if the first parameter is less than the second parameter, 
        #     it stores 1 in the position given by the third parameter. 
        #     Otherwise, it stores 0.
            if (get_posorval(instruction, 3, i+1, prog) < get_posorval(instruction, 2, i+2, prog)  ):
                if (assignparam == 0 ):
                    prog[prog[i+3]]=1
                elif(assignparam == 1 ):
                    prog[i+3]=1
            else:
                if (assignparam == 0 ):
                    prog[prog[i+3]]=0
                elif(assignparam == 1 ):
                    prog[i+3]=0
            i=i+4
        elif (opcode == 8):
        # Opcode 8 is equals: 
        # if the first parameter is equal to the second parameter, 
        #     it stores 1 in the position given by the third parameter. 
        #     Otherwise, it stores 0.
            if (get_posorval(instruction, 3, i+1, prog) == get_posorval(instruction, 2, i+2, prog)  ):
                if (assignparam == 0 ):
                    prog[prog[i+3]]=1
                elif(assignparam == 1 ):
                    prog[i+3]=1
            else:
                if (assignparam == 0 ):
                    prog[prog[i+3]]=0
                elif(assignparam == 1 ):
                    prog[i+3]=0
            i=i+4
    return output, prog

In [5]:
def guessables(maxnum):
    '''creates a list of all 5 digit numbers wherest every
    element has no repeating digits inside of that number+
    it doesn't count as a 5 digit number if it starts with a 0'''
    guesses=[]
    for a in range(0,maxnum):
        for b in range(0,maxnum):
            if a!=b:
                for c in range(0,maxnum):
                    if b!=c and a!=c:
                        for d in range(0,maxnum):
                            if c!=d and d!=b and d!=a:
                                for e in range(0,maxnum):
                                    if c!=e and b!=e and a!=e and d!=e:
                                        guesses.append(str(a)+str(b)+str(c)+str(d)+str(e))
    return guesses

In [6]:
def amp_circ(inputprog):
    '''
    list - > (str, int)
    
    Calculate the phase settings that maximise the output of the amplification circuits.
    returns phase settings and max value.
    
    >>> amp_circ([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0])
    (43210, '43210')
    >>> amp_circ([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0])
    (54321, '01234')
    >>> amp_circ([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0])
    (65210, '10432')
    '''
    #43210
    results={}
    for num in guessables(5):
        count=0
        x=0
        for i in num:
            i=int(i)
            (x,oldprog)=opcode(inputprog,i,x)
        results[num]=x
    maxkey=max(results.items(), key=operator.itemgetter(1))[0]

    return int(results[maxkey]), maxkey 
        

In [7]:
inputprog=[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
amp_circ(inputprog)

(43210, '43210')

In [8]:
amp_circ([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0])

(65210, '10432')

In [9]:
amp_circ([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0])

(43210, '43210')

In [10]:
amp_circ([3,8,1001,8,10,8,105,1,0,0,21,42,51,60,77,94,175,256,337,418,99999,3,9,1001,9,4,9,102,5,9,9,1001,9,3,9,102,5,9,9,4,9,99,3,9,102,2,9,9,4,9,99,3,9,1001,9,3,9,4,9,99,3,9,101,4,9,9,1002,9,4,9,101,5,9,9,4,9,99,3,9,1002,9,5,9,101,3,9,9,102,2,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,99])

(18812, '23041')

## Part Two


In [11]:
def guessables(minnum, maxnum):
    '''creates a list of all 5 digit numbers wherest every
    element has no repeating digits inside of that number+
    it doesn't count as a 5 digit number if it starts with a 0'''
    guesses=[]
    maxnum=maxnum+1
    for a in range(minnum,maxnum):
        for b in range(minnum,maxnum):
            if a!=b:
                for c in range(minnum,maxnum):
                    if b!=c and a!=c:
                        for d in range(minnum,maxnum):
                            if c!=d and d!=b and d!=a:
                                for e in range(minnum,maxnum):
                                    if c!=e and b!=e and a!=e and d!=e:
                                        guesses.append(str(a)+str(b)+str(c)+str(d)+str(e))
    return guesses

In [72]:
def opcode(prog, phase, inputval, i=0):
    '''
    list -> list
    
    A turing tape like computing device
    
    >>> opcode([1,0,0,0,99])
    [2, 0, 0, 0, 99]
    >>> opcode([2,3,0,3,99]) 
    [2, 3, 0, 6, 99]
    >>> opcode([2,4,4,5,99,0])
    [2, 4, 4, 5, 99, 9801]
    >>> opcode([1,1,1,4,99,5,6,0,99])
    [30, 1, 1, 4, 2, 5, 6, 0, 99]
    >>> opcode([4, 1, 99])
    1
    [4, 1, 99]
    >>> opcode([8,1,1,5,99,-1])
    [8, 1, 1, 5, 99, 1]
    >>> opcode([8,1,2,5,99,-1])
    [8, 1, 2, 5, 99, 0]
    >>> opcode([11008,1,1,5,99,-1])
    [11008, 1, 1, 1, 99, -1]
    >>> opcode([11108,1,2,5,99,-1])
    [11108, 1, 2, 0, 99, -1]
    '''
    inputnum=0
    opcode='00'
    while (opcode != 99):
        instruction=prog[i]
        assignparam=int(str(instruction).zfill(5)[0])
        opcode=get_opcode(instruction)
        if (opcode == 1):
            if (assignparam == 0 ):
                prog[prog[i+3]]=get_posorval(instruction, 3, i+1, prog)+get_posorval(instruction, 2, i+2, prog)
            elif(assignparam == 1 ):
                prog[i+3]=get_posorval(instruction, 3, i+1, prog)+get_posorval(instruction, 2, i+2, prog)
            i=i+4
        elif (opcode == 2):
            if (assignparam == 0 ):
                prog[prog[i+3]]=get_posorval(instruction, 3, i+1, prog)*get_posorval(instruction, 2, i+2, prog)
            elif(assignparam == 1 ):
                prog[i+3]=get_posorval(instruction, 3, i+1, prog)*get_posorval(instruction, 2, i+2, prog)
            i=i+4
        elif (opcode == 3):
        # Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. 
        # For example, the instruction 3,50 would take an input value and store it at address 50.
            assignparam=(instruction//100)%10
            inputnum=inputnum+1
            # Disable blocking input for now.
            #input_int=int(input())
            if (inputnum == 1):
                input_int=phase
            elif(inputnum ==2 ):
                input_int=inputval
            if (assignparam == 0 ):
                prog[prog[i+1]]=input_int
            elif(assignparam == 1 ):
                prog[i+1]=input_int
            i=i+2
        elif (opcode == 4):
        # Opcode 4 outputs the value of its only parameter. 
        # For example, the instruction 4,50 would output the value at address 50.
            assignparam=(instruction//100)%10
            if (assignparam == 0 ):
                output=prog[prog[i+1]]
                
            elif(assignparam == 1 ):
                output=prog[i+1]
            # Disable blocking input for now.

            #print(output)
            i=i+2
        elif (opcode == 5):
        # Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
            assignparam=(instruction//100)%10
            if (get_posorval(instruction, 3, i+1, prog) != 0 ):
                i = get_posorval(instruction, 2, i+2, prog)
            else:
                i=i+3
        elif (opcode == 6):
        # Opcode 6 is jump-if-false: 
        # if the first parameter is zero, 
        #     it sets the instruction pointer to the value from the second parameter. 
        #     Otherwise, it does nothing.
            assignparam=(instruction//100)%10
            if (get_posorval(instruction, 3, i+1, prog) == 0 ):
                i = get_posorval(instruction, 2, i+2, prog)
            else:
                i=i+3
        elif (opcode == 7):
        # Opcode 7 is less than: 
        # if the first parameter is less than the second parameter, 
        #     it stores 1 in the position given by the third parameter. 
        #     Otherwise, it stores 0.
            if (get_posorval(instruction, 3, i+1, prog) < get_posorval(instruction, 2, i+2, prog)  ):
                if (assignparam == 0 ):
                    prog[prog[i+3]]=1
                elif(assignparam == 1 ):
                    prog[i+3]=1
            else:
                if (assignparam == 0 ):
                    prog[prog[i+3]]=0
                elif(assignparam == 1 ):
                    prog[i+3]=0
            i=i+4
        elif (opcode == 8):
        # Opcode 8 is equals: 
        # if the first parameter is equal to the second parameter, 
        #     it stores 1 in the position given by the third parameter. 
        #     Otherwise, it stores 0.
            if (get_posorval(instruction, 3, i+1, prog) == get_posorval(instruction, 2, i+2, prog)  ):
                if (assignparam == 0 ):
                    prog[prog[i+3]]=1
                elif(assignparam == 1 ):
                    prog[i+3]=1
            else:
                if (assignparam == 0 ):
                    prog[prog[i+3]]=0
                elif(assignparam == 1 ):
                    prog[i+3]=0
            i=i+4
    if (opcode == 99 ):     
        return output, prog, i
    else:
        print ("ERROR: No exit code detected")
        return None

In [13]:
def recurs_amp_circ(inputprog):
    '''
    list - > (str, int)
    
    Calculate the phase settings that maximise the output of the amplification circuits.
    returns phase settings and max value.
    
    >>> recurs_amp_circ([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])
    (139629729, '98765')
    >>> recurs_amp_circ([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10])
    (18216, '97856')

    '''
    inputprog0=inputprog[:]
    inputprog1=inputprog[:]
    inputprog2=inputprog[:]
    inputprog3=inputprog[:]
    inputprog4=inputprog[:]
    #43210
    results={}
    for num in guessables(5,9):
        count=0
        x=0
        ans=-1
        for i in num:
            i=int(i)
            (x,oldprog)=opcode(inputprog3,i,x)
        results[num]=x
    maxkey=max(results.items(), key=operator.itemgetter(1))[0]

    return int(results[maxkey]), maxkey , results
        

In [16]:
def recurs_amp_circ(inputprog):
    '''
    list - > (str, int)
    
    Calculate the phase settings that maximise the output of the amplification circuits.
    returns phase settings and max value.
    
    >>> recurs_amp_circ([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])
    (139629729, '98765')
    >>> recurs_amp_circ([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10])
    (18216, '97856')

    '''
    #43210
    orig_prog=inputprog[:]
    results={}
    num=str(98765)
    count=0
    x=0
    ans=-1

    (x,oldprog0,pos0)=opcode(inputprog0,i,x) 
    (x,oldprog1,pos1)=opcode(inputprog0,i,x)  
    (x,oldprog2,pos2)=opcode(inputprog0,i,x)  
    (x,oldprog3,pos3)=opcode(inputprog0,i,x)  
    (x,oldprog4,pos4)=opcode(inputprog0,i,x) 
    
    (x,oldprog0,pos0)=opcode(inputprog0,i,x)  
    (x,oldprog0,pos0)=opcode(inputprog0,i,x)  
    (x,oldprog0,pos0)=opcode(inputprog0,i,x)  
    (x,oldprog0,pos0)=opcode(inputprog0,i,x)  

            
            
            
            
            
    results[num]=x
    maxkey=max(results.items(), key=operator.itemgetter(1))[0]
    return int(results[maxkey]), maxkey 

In [74]:
inputprog=[3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
prog0=inputprog[:]
prog1=inputprog[:]
prog2=inputprog[:]
prog3=inputprog[:]
prog4=inputprog[:]

results={}
num=str(98765)
count=0
x=0
ans=-1
amp=0
print (prog0)
(x, prog0, i0) = opcode(prog0,int(num[0]),x)
print (prog0)
(x, prog1, i1) = opcode(prog1,int(num[1]),x)
(x, prog2, i2) = opcode(prog2,int(num[2]),x)
(x, prog3, i3) = opcode(prog3,int(num[3]),x)
(x, prog4, i4) = opcode(prog4,int(num[4]),x)
print (prog0,int(num[0]),x,i0)
(x, prog0, i5) = opcode(prog0,int(num[0]),x,i0)
(x, prog1, i6) = opcode(prog1,int(num[1]),x,i1)
(x, prog2, i7) = opcode(prog2,int(num[2]),x,i2)


[3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26, 27, 4, 27, 1001, 28, -1, 28, 1005, 28, 6, 99, 0, 0, 5]
[3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26, 27, 4, 27, 1001, 28, -1, 28, 1005, 28, 6, 99, 5, 5, 0]
[3, 26, 1001, 26, -4, 26, 3, 27, 1002, 27, 2, 27, 1, 27, 26, 27, 4, 27, 1001, 28, -1, 28, 1005, 28, 6, 99, 5, 5, 0] 9 129 25


UnboundLocalError: local variable 'output' referenced before assignment

In [ ]:
type(x)

In [18]:
a = recurs_amp_circ([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])

NameError: name 'inputprog0' is not defined

In [119]:
print(a)

None


In [26]:
input_prog=[3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

In [33]:
old_prog=input_prog[:]
old_prog1=input_prog[:]
old_prog2=input_prog[:]

In [34]:
old_prog[0]=99999
old_prog1[0]=-123456
old_prog2[0]='00000'

In [37]:
old_prog2

['00000',
 26,
 1001,
 26,
 -4,
 26,
 3,
 27,
 1002,
 27,
 2,
 27,
 1,
 27,
 26,
 27,
 4,
 27,
 1001,
 28,
 -1,
 28,
 1005,
 28,
 6,
 99,
 0,
 0,
 5]